As of now i am just make a classification model in just one ipynb not in multiple modules. I will use pytorch for classification.

In [ ]:
import torch
import torch.nn as nn
impochrt torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

loading the data and splitting the data into the train and the test set before doing that we will verify our image datasets if encounter any corroupted file or broken image file we will remove it.

In [2]:
import os
from PIL import Image

data_dir = "kagglecatsanddogs_3367a\PetImages"
classes = ["Cat","Dog"]

for cls in classes:
    cls_path = os.path.join(data_dir,cls)
    for img in os.listdir(cls_path):
        img_path=os.path.join(cls_path,img)
        try:
            with Image.open(img_path) as im:
                im.verify()
                
        except:
            print(f"this file is corroupted : {img_path}")
            os.remove(img_path)
        

c:\Users\Pallavi\OneDrive\Documents\computer vision\venv\lib\site-packages\PIL\TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


we fetched out a corroupted file out of the dataset now we can split our data. we will be using os as always 🗿 and shutil to make robust copy of data from one loc to other.

In [3]:
import os,shutil,random

root_dir = "kagglecatsanddogs_3367a\PetImages"
dest="Data"
classes = ["Cat","Dog"]
split=0.8

for cls in classes:
    images = os.listdir(os.path.join(root_dir,cls))
    random.shuffle(images)
    
    train_cut = int(len(images)*split)
    
    train_list = images[:train_cut]
    test_list = images[train_cut:]
    
    for phase,imgs in [("train",train_list),("test",test_list)]:
        out_dir = os.path.join(dest,phase,cls)
        os.makedirs(out_dir,exist_ok=True)
        for img in imgs:
            shutil.copy(
                os.path.join(root_dir,cls,img),
                os.path.join(out_dir,img)
            )
    

loading and tranforming the data for the image classification.

In [4]:
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

train_ds = datasets.ImageFolder("data/train",transform=transform)
test_ds = datasets.ImageFolder("data/test",transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader   = DataLoader(train_ds, batch_size=32, shuffle=False)

I SPLITTED THE DATA INTO TRAIN AND THE TEST SET AND ALSO REDUCED ALL THE IMAGE SIZE TO AVOID ANY BUG. NOW I AM GONNA MAKE A SIMPLE CNN MODEL WHICH GONNA CLASSIFY THE GIVEN PICTURE IS DOG OR OF CAT.

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, 3, activation='relu' , input_shape=(128,128,3)),
    MaxPooling2D(2),
    
    Conv2D(64,3,activation='relu'),
    MaxPooling2D(2),
    
    Conv2D(128,3,activation='relu'),
    MaxPooling2D(2),
    
    Flatten(),
    Dense(128,activation='relu'),
    Dense(1,activation='sigmoid')
    
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    
)

model.summary()

c:\Users\Pallavi\OneDrive\Documents\computer vision\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break


torch.Size([32, 3, 128, 128]) torch.Size([32])


SO AS I CAN SEE HERE IF I WANT TO FIT THIS DATA FIRST OF ALL I NEED TO CONVERT WE HAVE CHW BUT WE NEED HWC AND WE ARE CONVERTING THIS TENSOR INTO NUMPY ARRAY BECAUSE KERAS EXPECT NUMPY ARRAYS.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2   # 👈 BEST PRACTICE
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = train_datagen.flow_from_directory(
    'data/train',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 24916 images belonging to 2 classes.


In [ ]:
val_gen = train_datagen.flow_from_directory(
    'data/train',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


NameError: name 'val_gen' is not defined